In [4]:
import qcomp as Q

# Deutch's problem

Reference: Quantum algorithms revisited, R Cleve et al https://arxiv.org/abs/quant-ph/9708016

## problem statement

Given single bit function f that returns {0,1}, on single run, determine whether it is balanced or constant?
balanced: function will return equal number of 0s and 1s. Two versions

1. $f(0)=0 & f(1)=1$
2. $f(0)=1 & f(1)=0$

constant: function will return always 0 or 1. There are two possibilities 

1. $f(0)=0 & f(1)=0$
2. $f(1)=1 & f(1)=1$

## solution

Classically impossible, however, by using following quantum circuit it can be solved.

![Q circuit](images/deutch200.png "Deutch's solution, has HfH form")

Here, $U_f$ is unitary transformation matrix form of function. Upper qbit is the input, lower one is output. When lower is set to 0, after transformation it will be value of function. If lower bit is set to 1, its value will be $1+f(x) \mod 2$ instead. The magic happens when input is $|0>(|0>-|1>)$ which turns into entangled (|0>+|1>)(|0>-|1>) after first Hadamar gate. After $U_f$ and second Hadamar gate is applied, first (input) bit will be 0 if $f$ is constant, 1 if $f$ is balanced! So that is first qbit we are interested.

Note that we can not learn actual value of function in inputs, which is tradeoff being made. And it also explains uncertainty principle in some sense, by not knowing real value, we can learn about properties of function.

### implementation

$U_f$ can be prepared by using *cnot gate*

In [6]:
# balanced function will be reversing input 0->1, 1->0
# so we want this
# 0 -> not ->  1  -> not -> 0
#              |
# 0 --------> not --------> 1
# 1 -> not -> 0   -> not  ->1
#             |   
# 0---------> not --------->0
from qcomp.gate import *
# firt make not gates acting on first qubit
Ub = (Q.gate.NOT*Q.gate.I) + (CNOT) + (Q.gate.NOT*Q.gate.I)

# let us also make constant function that only produces 1
# this is just making not gate act on 2nd register regardless of 1
# not on 2nd qbit
Uc = Q.gate.I * Q.gate.NOT

Sanity check

In [7]:
from qcomp.qbit import ZERO,ONE
# inputs
Q00 = Q.qregister.mk_reg([ZERO,ZERO])
Q10 = Q.qregister.mk_reg([ONE,ZERO])

In [9]:
# calculate outputs
print("Ub on input 0 -> {}".format(Ub.apply(Q00).get_qbit(1)[0]))
print("Ub on input 1 -> {}".format(Ub.apply(Q10).get_qbit(1)[0]))

Ub on input 0 -> 0j|0> + (1+0j)|1>
Ub on input 1 -> (1+0j)|0> + 0j|1>


In [10]:
print("Uc on input 0 -> {}".format(Uc.apply(Q00).get_qbit(1)[0]))
print("Uc on input 1 -> {}".format(Uc.apply(Q10).get_qbit(1)[0]))

Uc on input 0 -> 0j|0> + (1+0j)|1>
Uc on input 1 -> 0j|0> + (1+0j)|1>


So, Ub produces state 1 in output bit when input is 0 and vice-versa as expected. And Uc produces state 1 all the time.

### Algorithm circuit

In [11]:
HI = H*I

In [12]:
deutch_Ub = HI+Ub+HI
deutch_Uc = HI+Uc+HI

In [13]:
# magical input is |0>-|1> which is defined as MINUS  in our code
from qcomp.qbit import MINUS
from qcomp.qregister import mk_reg
deutch_in = mk_reg([ZERO,MINUS])
deutch_in.state

array([ 0.7071068+0.j, -0.7071068+0.j,  0.       +0.j,  0.       +0.j],
      dtype=complex64)

In [14]:
# bit we are interested is upper one, ie 0th index
print("Ub produces {}".format(deutch_Ub.apply(deutch_in).get_qbit(0)))
print("Uc produces {}".format(deutch_Uc.apply(deutch_in).get_qbit(0)))

Ub produces ('0j|0> + (1.0000001192092896+0j)|1>', <qcomp.qbit.QBit object at 0x7fe0d031bbe0>)
Uc produces ('(1.0000001192092896+0j)|0> + 0j|1>', <qcomp.qbit.QBit object at 0x7fe0d031bcc0>)


constant function produced 0 state, balanced one produced 1 as promised

## Generalized form

More general form of is where function can take input of n bits ${0,1}^n$ and produce 1 or 0. Again question is same, but we are also promised that it is either balanced or constant. Solution has same form, Hadamard gates are applied to all input bits. If function is balanced amplitude of first n bits being 0, ie state $|00..0>(output)$ will be zero, otherwise non-zero (actually can be calculated).\\

> **Unfortunately it is imposssible to construct arbitrary function unless you wanna get dirty with CNOT, NOT gates and make complicated circuits!!! Once MultiCNOT is implemented it should be relatively easy. On the other hand** *construct_unitary_F* **works fine for now**

Let us make two 3bit input function, one balanced, one constant

In [15]:
from qcomp.gate import construct_unitary_F

balanced3 = ["010","110","111","001"]
constant3 = [] # always produce 0

Ub3 = construct_unitary_F(balanced3)
Uc3 = construct_unitary_F(constant3, in_size=3)

Testing if function work properly

In [16]:
Q0100 = mk_reg([ZERO,ONE,ZERO,ZERO])  # output->1
Q1110 = mk_reg([ONE,ONE,ONE,ZERO])    # output->1
Q0000 = mk_reg([ZERO,ZERO,ZERO,ZERO]) # output->0
Q1100 = mk_reg([ONE,ONE,ZERO,ZERO])   # output->1
Q1000 = mk_reg([ONE,ZERO,ZERO,ZERO])  # output->0

In [17]:
# Note output bit is 3rd
print("Ub3 on input 010 -> {}".format(Ub3.apply(Q0100).get_qbit(3)))
print("Ub3 on input 111 -> {}".format(Ub3.apply(Q1110).get_qbit(3)))
print("Ub3 on input 000 -> {}".format(Ub3.apply(Q0000).get_qbit(3)))
print("Ub3 on input 110 -> {}".format(Ub3.apply(Q1100).get_qbit(3)))
print("Ub3 on input 100 -> {}".format(Ub3.apply(Q1000).get_qbit(3)))

Ub3 on input 010 -> ('0j|0> + (1+0j)|1>', <qcomp.qbit.QBit object at 0x7fe0d032aac8>)
Ub3 on input 111 -> ('0j|0> + (1+0j)|1>', <qcomp.qbit.QBit object at 0x7fe0d032add8>)
Ub3 on input 000 -> ('(1+0j)|0> + 0j|1>', <qcomp.qbit.QBit object at 0x7fe0d032ad68>)
Ub3 on input 110 -> ('0j|0> + (1+0j)|1>', <qcomp.qbit.QBit object at 0x7fe0d032acf8>)
Ub3 on input 100 -> ('(1+0j)|0> + 0j|1>', <qcomp.qbit.QBit object at 0x7fe0d032ac88>)


In [18]:
# constant function will always produce 0
print("Uc3 on input 010 -> {}".format(Uc3.apply(Q0100).get_qbit(3)))
print("Uc3 on input 111 -> {}".format(Uc3.apply(Q1110).get_qbit(3)))
print("Uc3 on input 000 -> {}".format(Uc3.apply(Q0000).get_qbit(3)))
print("Uc3 on input 110 -> {}".format(Uc3.apply(Q1100).get_qbit(3)))
print("Uc3 on input 100 -> {}".format(Uc3.apply(Q1000).get_qbit(3)))

Uc3 on input 010 -> ('(1+0j)|0> + 0j|1>', <qcomp.qbit.QBit object at 0x7fe0d032ac18>)
Uc3 on input 111 -> ('(1+0j)|0> + 0j|1>', <qcomp.qbit.QBit object at 0x7fe0d032ab38>)
Uc3 on input 000 -> ('(1+0j)|0> + 0j|1>', <qcomp.qbit.QBit object at 0x7fe0d032ae10>)
Uc3 on input 110 -> ('(1+0j)|0> + 0j|1>', <qcomp.qbit.QBit object at 0x7fe0d032acc0>)
Uc3 on input 100 -> ('(1+0j)|0> + 0j|1>', <qcomp.qbit.QBit object at 0x7fe0d032ae48>)


### Algorithm

In [19]:
# 3 input, 1 output, we need 3 Hadamard one ID
HHHI = H**3 * I

In [20]:
deutch_Ub3 = HHHI+Ub3+HHHI
deutch_Uc3 = HHHI+Uc3+HHHI

In [21]:
deutch_in3 = mk_reg([ZERO,ZERO,ZERO,MINUS])

Unfortunatly I have not implemented a method to get state of specific subset of bits. However, we are interested in amplitude of first 3 bits being 0, ie states $|0000>$ and $|0001>$. Therefore, we can determine by looking at first two coefficients of resulting register. 

In [22]:
res_constant = deutch_Uc3.apply(deutch_in3)
print("Result of constant function is \n {} \n".format(res_constant))
print("First two bases {}".format(res_constant.state[:2]))

Result of constant function is 
 0.71+0.00j 	 |0000> 
-0.71+0.00j 	 |0001> 
0.00+0.00j 	 |0010> 
0.00+0.00j 	 |0011> 
0.00+0.00j 	 |0100> 
0.00+0.00j 	 |0101> 
0.00+0.00j 	 |0110> 
0.00+0.00j 	 |0111> 
-0.00+0.00j 	 |1000> 
0.00+0.00j 	 |1001> 
0.00+0.00j 	 |1010> 
0.00+0.00j 	 |1011> 
0.00+0.00j 	 |1100> 
0.00+0.00j 	 |1101> 
0.00+0.00j 	 |1110> 
0.00+0.00j 	 |1111> 
 

First two bases [ 0.7071068+0.j -0.7071068+0.j]


In [23]:
res_balanced = deutch_Ub3.apply(deutch_in3)
print("Result of balanced function is \n {} \n".format(res_balanced))
print("First two bases {}".format(res_balanced.state[:2]))

Result of balanced function is 
 0.00+0.00j 	 |0000> 
0.00+0.00j 	 |0001> 
0.00+0.00j 	 |0010> 
0.00+0.00j 	 |0011> 
0.35+0.00j 	 |0100> 
-0.35+0.00j 	 |0101> 
0.35+0.00j 	 |0110> 
-0.35+0.00j 	 |0111> 
0.00+0.00j 	 |1000> 
0.00+0.00j 	 |1001> 
0.00+0.00j 	 |1010> 
0.00+0.00j 	 |1011> 
-0.35+0.00j 	 |1100> 
0.35+0.00j 	 |1101> 
0.35+0.00j 	 |1110> 
-0.35+0.00j 	 |1111> 
 

First two bases [0.+0.j 0.+0.j]


As promised, balanced function gave amplitude of 0!

# Relevance to Grover's

While Deutch's is rather peculiar algorithm and certainly not part of course, the idea of constructing unitary transformation matrix is shared in both. Since this one is much easier to make and nice playground for testing implementations